## 准备数据

In [18]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    return (x, y), (x_test, y_test)

In [19]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [20]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal(shape=(28 * 28, 512), stddev=0.05), trainable=True)
        self.b1 = tf.Variable(tf.zeros(shape=(512,)), trainable=True)
        self.W2 = tf.Variable(tf.random.normal(shape=(512, 10), stddev=0.05), trainable=True)
        self.b2 = tf.Variable(tf.zeros(shape=(10,)), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_flattened = tf.reshape(x, [-1, 784])
        h1 = tf.matmul(x_flattened, self.W1) + self.b1
        activated_h1 = tf.tanh(h1)
        logits = tf.matmul(activated_h1, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [21]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [22]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3143072 ; accuracy 0.1269
epoch 1 : loss 2.2918127 ; accuracy 0.13778333
epoch 2 : loss 2.2699811 ; accuracy 0.1502
epoch 3 : loss 2.2487552 ; accuracy 0.164
epoch 4 : loss 2.2280843 ; accuracy 0.17986667
epoch 5 : loss 2.2079265 ; accuracy 0.19736667
epoch 6 : loss 2.1882427 ; accuracy 0.21443333
epoch 7 : loss 2.1690006 ; accuracy 0.23208334
epoch 8 : loss 2.1501703 ; accuracy 0.24988334
epoch 9 : loss 2.1317267 ; accuracy 0.26783332
epoch 10 : loss 2.113647 ; accuracy 0.2848
epoch 11 : loss 2.0959108 ; accuracy 0.3014
epoch 12 : loss 2.0784998 ; accuracy 0.31955
epoch 13 : loss 2.0613985 ; accuracy 0.33598334
epoch 14 : loss 2.0445924 ; accuracy 0.35206667
epoch 15 : loss 2.0280685 ; accuracy 0.36733332
epoch 16 : loss 2.0118153 ; accuracy 0.3813
epoch 17 : loss 1.9958224 ; accuracy 0.39461666
epoch 18 : loss 1.9800806 ; accuracy 0.40956667
epoch 19 : loss 1.964581 ; accuracy 0.42258334
epoch 20 : loss 1.9493161 ; accuracy 0.43538332
epoch 21 : loss 1.934279 ; accur